# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверка четырёх гипотез:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

## Обзор данных

In [7]:
import pandas as pd
data = pd.read_csv('datasets/bank.csv')
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Даже на первый взгляд видно множество косяков в данных. Надо исправлять!

## Обработка данных

### Заполнение пропусков

Судя по данным, выведенным функцией info(), пропуски есть только в 'days_employed' и 'total_income'. От этого и будем отталкиваться. Для начала взглянем на долю пустых значений в этих столбцах.

In [8]:
isna_1 = data['days_employed'].isna().sum()
isna_2 = data['total_income'].isna().sum()
print('Доля пустых значений в столбце days_employed равна', str(int(isna_1 / 21525*100))+'%')
print('Доля пустых значений в столбце total_income  равна', str(int(isna_2 / 21525*100))+'%')

Доля пустых значений в столбце days_employed равна 10%
Доля пустых значений в столбце total_income  равна 10%


Чтож, их доля крайне немаленькая, давайте взглянем на данные поближе

In [9]:
print(data['days_employed'].head(15))

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64


Судя по всему при внесении данных в столбец 'days_employed' были совершены технические ошибки, такие как:

- неправильной постановки точки, отделяющей целую и дробную часть значения или другая ошибка при внесении, повлекшая образование аномальных значений (например 340266.072047)
- внесение минусовых значений
- а также некоторые значения попросту отсутствуют

Я сделал вывод о технической причине возникновения ошибок, так как думаю что только тех ошибка могла повлечь образование столь огромного количества отрицательных значений в датасете.

Последние 2 ошибки сейчас мы и будем исправлять.

In [15]:
data['days_employed'] = abs(data['days_employed'])

list = ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий']
for k in list:
    median = (data[data['income_type'] == k])['days_employed'].median()
    data.loc[(data['income_type'] == k), 'days_employed'].fillna(median)

data = data.dropna(subset = ['days_employed'])
print(data['days_employed'].isna().sum())

0


Таким образом мы убрали отрицательные и заполнили пустые значения медианным по категориям 'income_type', так как аномальные значения могут сильно испортить статистику если мы будем использовать median() для заполнения пустых значений

In [7]:
print(data['total_income'].head(15))
print(data['total_income'].min())
print(data['total_income'].max())

0     253875.639453
1     112080.014102
2     145885.952297
3     267628.550329
4     158616.077870
5     255763.565419
6     240525.971920
7     135823.934197
8      95856.832424
9     144425.938277
10    113943.491460
11     77069.234271
13    130458.228857
14    165127.911772
15    116820.904450
Name: total_income, dtype: float64
20667.26379327158
2265604.028722744


Теперь переключимся на столбец 'total_income'

На первый взгляд видны только пропуски. Нет отрицательных или нереалестично завышенных значений. Зарплаты в 20тыс и 2,2млн вполне реальны. Хотя немного непонятно зачем и на что надо брать кредит, имея ежемесячную зарплату в 2,2 млн, ну да ладно.

Заполним пропуски в столбце точно также как и в прошлый раз.

In [8]:
list = ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий']
for k in list:
    median = (data[data['income_type'] == k])['total_income'].median()
    data.loc[(data['income_type'] == k), 'total_income'].fillna(median)

print (data['total_income'].isna().sum())

0


Готово, проверим наличие пропусков через функцию info()

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


**Вывод:**

Были обнаружены и заполнены пропущенные значения в 2 столбцах: 'days_employed' и 'total_income'.

Невероятные значения в столбце 'days_employed' скорее похожи на техническую ошибку,
в то время как в 'total_income' напротив на человеческий фактор.

Заполнялись пропущенные значения медианным значением, для каждого 'income_type' значение своё

### Проверка данных на аномалии и исправления.

Пропуски в 'days_employed' и 'total_income' были устранены. Как и отрицательные значения в столбце 'days_employed'.
Но надо проверить остальные столбцы на возможное наличие аномальных значений.

In [11]:
print (data['children'].unique())
print (data[data['children'] == -1]['children'].count())
print (data[data['children'] == 20]['children'].count())

print (sorted(data['dob_years'].unique()))
print (data[data['dob_years'] == 0]['dob_years'].count())

print (data['gender'].unique())
print (data[data['gender'] == 'XNA']['gender'].count())

[ 1  0  3  2 -1  4 20  5]
44
67
[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
91
['F' 'M' 'XNA']
1


Видим странные значения в столбце 'children': -1 и 20. Наличие 20 детей конечно возможно, но отсутствие клиентов с 6-19 детьми и 21+ и тот факт, что это далеко не единичный случай (их аж 76!) ставит под сомнения корректность данных. Вероятнее всего в появлении '-1' и '20' виноват человеческий фактор. Решение: '-1' надо переправить на '1', а '20' на '2'.

In [12]:
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20, 2)
print (data['children'].unique())

[1 0 3 2 4 5]


Готово! Переходим к следующему столбцу - 'dob_years'. И тут видим странное значение - 0 лет. Сомнительно, что банк готов дать кредит несовершеннолетнему, к тому же которому 0 лет! Тут уже сомнительна человеческая ошибка, так как таких значений аж 101!(если только отвественные за внесение данных люди не ставят "0" при отсутствии информации о возрасте клиентов).
Для начала у меня появилась гипотеза, что произошла техническая ошибка, из-за которой условно год "40" стал отображаться во всем столбце как "0". Но, после проверки уникальных значений через unique() и sorted() я пришел к выводу, что гипотеза не верна, и решил заменить "0" на медиану столбца 'dob_years'

In [13]:
#print(data['income_type'].unique())
print (data['income_type'].value_counts())

#print(data[data['income_type'] == 'безработный']['dob_years'])
#print(data[data['income_type'] == 'предприниматель']['dob_years'])
#print(data[data['income_type'] == 'студент']['dob_years'])
#print(data[data['income_type'] == 'в декрете']['dob_years'])
#как видим, данные категории не нуждаются в замене значения 'dob_years'

type_1 = int((data[data['income_type'] == 'сотрудник'])['dob_years'].mean())
type_2 = int((data[data['income_type'] == 'пенсионер'])['dob_years'].mean())
type_3 = int((data[data['income_type'] == 'компаньон'])['dob_years'].mean())
type_4 = int((data[data['income_type'] == 'госслужащий'])['dob_years'].mean())

data.loc[(data['income_type'] == 'сотрудник') & (data['dob_years'] == 0), 'dob_years'] = type_1
data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] == 0), 'dob_years'] = type_2
data.loc[(data['income_type'] == 'компаньон') & (data['dob_years'] == 0), 'dob_years'] = type_3
data.loc[(data['income_type'] == 'госслужащий') & (data['dob_years'] == 0), 'dob_years'] = type_4

print(data[data['dob_years'] == 0]['dob_years'].count())

сотрудник          10014
компаньон           4577
пенсионер           3443
госслужащий         1312
безработный            2
в декрете              1
студент                1
предприниматель        1
Name: income_type, dtype: int64
0


Готово. Теперь переходим к последнему артефакту - единственному значению 'XNA' в столбце 'gender'.
Думаю не будет критичным сохранить эту строку, просто поменяв на один из полов, особого значения при последующем анализе данных не играет на какой.

In [14]:
data['gender'] = data['gender'].replace('XNA', 'M')
print (data['gender'].unique())

['F' 'M']


### Изменение типов данных.

In [15]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


Используя функцию astype() заменяем тип данных для столбцов 'days_employed' и 'total_income' с вещественного (float) на целочисленный (int)

### Удаление дубликатов.

In [16]:
print (data['education'].unique())
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data = data.drop_duplicates().reset_index(drop=True)
print (data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Для начала мы избавляемся от явных дубликатов применяя функцию drop_duplicates(). И заодно обновляем индексы.

Далее мы смотрим какие уникальные значения встречаются в столбце 'education' и приводим все значения к нижниму регистру используя функцию lower()

Причиной возникновения дубликатов в столбце 'education' является человеческий фактор. Кто-то написал капс локом, кто-то начал с заглавной, кто-то напротив с строчной буквы. По этой причине и появилось так много неявных дубликатов в данных

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [17]:
education = pd.DataFrame(data, columns = {'education_id', 'education'})
family_status = pd.DataFrame(data, columns = {'family_status_id', 'family_status'})
print (education.head(10))
print (family_status.head(10))
data = data.drop(columns = ['education', 'family_status'], axis = 1)
print (data.head())

  education  education_id
0    высшее             0
1   среднее             1
2   среднее             1
3   среднее             1
4   среднее             1
5    высшее             0
6    высшее             0
7   среднее             1
8    высшее             0
9   среднее             1
   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем
   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1     

Для начала создаем 2 справочника: education и family_status и передаем им по 2 столбца из основной таблицы.
Затем удаляем столбцы 'education' и 'family_status' из основной таблицы.

### Шаг 2.6. Категоризация дохода.

In [18]:
#print('Минимальная зарплата:', data['total_income'].min())
#print('Максимальная зарплата:', data['total_income'].max())
#print('Средняя зарплата:', int(data['total_income'].mean()))
#print('Медиана:', int(data['total_income'].median()))

def salary(data):
    if data <= 30000:
        return 'E'
    if 30001 <= data <= 50000:
        return 'D'
    if 50001 <= data <= 200000:
        return 'C'
    if 200001 <= data <= 1000000:
        return 'B'
    else:
        return 'A'    


total_income_category = data['total_income'].apply(salary)
data.insert(9, 'total_income_category', total_income_category)
print(data.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0         340266         53             1                 1      F   
5         0            926         27             0                 1      M   
6         0           2879         43             0                 0      F   
7         0            152         50             1                 0      M   
8         2           6929         35             0                 1      F   
9         0           2188         41             1                 0      M   

  income_type  debt  total_income total_income_category  \
0   сотрудник     0        253875                     B   
1

Для начала создали функцию salary(), которая сортирует полученные данные и возвращает тип зарплаты. Затем применили salary() к столбцу total_income и внесли полученные значения в новую колонку 'total_income_category'

### Шаг 2.7. Категоризация целей кредита.

In [19]:
from pymystem3 import Mystem
m = Mystem()
def lem(text):
    lemmas = m.lemmatize(text)
    if 'автомобиль' in lemmas:
        return 'операции с автомобилем'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'операции с недвижимостью'
    if 'образование' in lemmas:
        return 'получение образования'
    if 'свадьба' in lemmas:
        return 'проведение свадьбы'
        
data['purpose_category'] = data['purpose'].apply(lem)
print (data.head())

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0         340266         53             1                 1      F   

  income_type  debt  total_income total_income_category  \
0   сотрудник     0        253875                     B   
1   сотрудник     0        112080                     C   
2   сотрудник     0        145885                     C   
3   сотрудник     0        267628                     B   
4   пенсионер     0        158616                     C   

                      purpose          purpose_category  
0               покупка жилья  операции с недвижимостью  
1     приобретение автомобиля    операции с авто

Сначала выявил основные категории целей взятия кредита (жилье/недвижимость, автомобиль, образование и свадьба)

Используя функцию lem() отсортировал при помощи лемматизации все цели взятия кредита по основным категориям.

Вдальнейшем это сильно пригодиться при формировании ответов на вопросы, тк обращаться к категориям гораздо сподручней

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
#print (data['children'].unique())
#print (data['children'].value_counts())

child_0 = data[data['children'] == 0]
child_1 = data[data['children'] == 1]
child_2 = data[data['children'] == 2]
child_3 = data[data['children'] == 3]
child_4 = data[data['children'] == 4]
child_5 = data[data['children'] == 5]

debt_child_0 = child_0[child_0['debt'] == 1]['debt'].count()
debt_child_1 = child_1[child_1['debt'] == 1]['debt'].count()
debt_child_2 = child_2[child_2['debt'] == 1]['debt'].count()
debt_child_3 = child_3[child_3['debt'] == 1]['debt'].count()
debt_child_4 = child_4[child_4['debt'] == 1]['debt'].count()
debt_child_5 = child_5[child_5['debt'] == 1]['debt'].count()

print ('Процент невозврата среди людей без детей:', 
       str(int(debt_child_0/child_0['children'].count()*100))+'%')
print ('Процент невозврата среди людей с 1 ребенком:', 
       str(int(debt_child_1/child_1['children'].count()*100))+'%')
print ('Процент невозврата среди людей с 2 детьми:', 
       str(int(debt_child_2/child_2['children'].count()*100))+'%')
print ('Процент невозврата среди людей с 3 детьми:', 
       str(int(debt_child_3/child_3['children'].count()*100))+'%')
print ('Процент невозврата среди людей с 4 детьми:', 
       str(int(debt_child_4/child_4['children'].count()*100))+'%')
print ('Процент невозврата среди людей с 5 детьми:', 
       str(int(debt_child_5/child_5['children'].count()*100))+'%')

Процент невозврата среди людей без детей: 7%
Процент невозврата среди людей с 1 ребенком: 9%
Процент невозврата среди людей с 2 детьми: 9%
Процент невозврата среди людей с 3 детьми: 7%
Процент невозврата среди людей с 4 детьми: 8%
Процент невозврата среди людей с 5 детьми: 0%


Для общего представления с помощью функции value_counts() смотрим в столбце 'children' сколько клиентов подподают под каждую категорию дохода.

Затем в новые переменные записываем строки с 6 категориями клиентов, имеющими от 0 до 5 детей. И, наконец, находим количество имевших задолжности и их отношение ко всему количеству в каждой из категорий клиентов.

##### Вывод:
Сложно говорить о какой-то закономерности, ведь среди людей без детей и людей с 1-4 детьми в среднем 7-9% имели задолженность по возврату кредита. Среди людей с 5 детьми эта статистика равна 0%, но, что немаловажно таких клиентов всего 9, и говорить о какой-то закономерности со столь малым количеством данных бессмысленно. Если бы процент имевших задолжность увеличивался или уменьшался по мере роста количества детей, то тогда можно было расценить это как некую закономерность. Но по факту её тут нет

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
#print (family_status['family_status'].value_counts())
#print (family_status['family_status_id'].value_counts())

status_0 = data[data['family_status_id'] == 0]
status_1 = data[data['family_status_id'] == 1]
status_2 = data[data['family_status_id'] == 2]
status_3 = data[data['family_status_id'] == 3]
status_4 = data[data['family_status_id'] == 4]

debt_status_0 = status_0[status_0['debt'] == 1]['debt'].count()
debt_status_1 = status_1[status_1['debt'] == 1]['debt'].count()
debt_status_2 = status_2[status_2['debt'] == 1]['debt'].count()
debt_status_3 = status_3[status_3['debt'] == 1]['debt'].count()
debt_status_4 = status_4[status_4['debt'] == 1]['debt'].count()

print ('Процент невозврата в категории "женат/замужем":      ', 
       str(int(debt_status_0/status_0['family_status_id'].count()*100))+'%')
print ('Процент невозврата в категории "гражданский брак":   ', 
       str(int(debt_status_1/status_1['family_status_id'].count()*100))+'%')
print ('Процент невозврата в категории "не женат/не замужем":', 
       str(int(debt_status_2/status_2['family_status_id'].count()*100))+'%')
print ('Процент невозврата в категории "в разводе":          ', 
       str(int(debt_status_3/status_3['family_status_id'].count()*100))+'%')
print ('Процент невозврата в категории "вдовец/вдова":       ', 
       str(int(debt_status_4/status_4['family_status_id'].count()*100))+'%')

Процент невозврата в категории "женат/замужем":       7%
Процент невозврата в категории "гражданский брак":    9%
Процент невозврата в категории "не женат/не замужем": 6%
Процент невозврата в категории "в разводе":           7%
Процент невозврата в категории "вдовец/вдова":        10%


Для общего представления с помощью функции value_counts() смотрим сколько значений в столбцах 'family_status_id' и 'family_status' для каждой категории. Делаем вывод какой идентификатор в 'family_status_id' к какой категории 'family_status' принадлежит.

Затем в новые переменные записываем строки с одной из 5 категорий семейного положения. И, наконец, находим количество имевших задолжности и их отношение ко всему количеству в каждой из категорий.

##### Вывод:
После проделанных расчетов для каждой категории семейного статуса можно с уверенностью сказать, что семейный статус не влияет на возврат кредита в срок, так как проценты невозвратов для каждой категории очень схожи и максимальная разница между ними не превышает 3%. Можно лишь подметить, что процент невозратов у категорий клиентов "гражданский брак" и "вдовец/вдова" чуть больше средних 6-7% и составляют 9% и 10% соответственно.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
#print(data['total_income_category'].value_counts())

income_e = data[data['total_income_category'] == 'E']
income_d = data[data['total_income_category'] == 'D']
income_c = data[data['total_income_category'] == 'C']
income_b = data[data['total_income_category'] == 'B']
income_a = data[data['total_income_category'] == 'A']

debt_income_e = income_e[income_e['debt'] == 1]['debt'].count()
debt_income_d = income_d[income_d['debt'] == 1]['debt'].count()
debt_income_c = income_c[income_c['debt'] == 1]['debt'].count()
debt_income_b = income_b[income_b['debt'] == 1]['debt'].count()
debt_income_a = income_a[income_a['debt'] == 1]['debt'].count()

print('Процент невозвратов в категории "E":',
      str(int(debt_income_e/income_e['total_income_category'].count()*100))+'%')
print('Процент невозвратов в категории "D":',
      str(int(debt_income_d/income_d['total_income_category'].count()*100))+'%')
print('Процент невозвратов в категории "C":',
      str(int(debt_income_c/income_c['total_income_category'].count()*100))+'%')
print('Процент невозвратов в категории "B":',
      str(int(debt_income_b/income_b['total_income_category'].count()*100))+'%')
print('Процент невозвратов в категории "A":',
      str(int(debt_income_a/income_a['total_income_category'].count()*100))+'%')

Процент невозвратов в категории "E": 9%
Процент невозвратов в категории "D": 6%
Процент невозвратов в категории "C": 8%
Процент невозвратов в категории "B": 7%
Процент невозвратов в категории "A": 8%


Для общего представления с помощью функции value_counts() смотрим в столбце 'total_income_category' сколько клиентов подподают под каждую категорию дохода.

Затем в новые переменные записываем строки с одной из 5 категорий уровня дохода. И, наконец, находим количество имевших задолжности и их отношение ко всему количеству в каждой из категорий.

##### Вывод:
После проделанных расчетов для каждой категории уровня дохода можно с уверенностью сказать, что уровень дохода не влияет на возврат кредита в срок, так как проценты невозвратов для каждой категории очень схожи и максимальная разница между ними не превышает 3%.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [23]:
#print (data['purpose_category'].value_counts())

house = data[data['purpose_category'] == 'операции с недвижимостью']
avto = data[data['purpose_category'] == 'операции с автомобилем']
education = data[data['purpose_category'] == 'получение образования']
wedding = data[data['purpose_category'] == 'проведение свадьбы']

house_debt = house[house['debt'] == 1]['debt']
avto_debt = avto[avto['debt'] == 1]['debt']
education_debt = education[education['debt'] == 1]['debt']
wedding_debt = wedding[wedding['debt'] == 1]['debt']

print ('Процент невозвратов в категории "Недвижимость":', 
       str(int(house_debt.count()/house['purpose_category'].count()*100))+'%')
print ('Процент невозвратов в категории "Автомобили"  :', 
       str(int(avto_debt.count()/avto['purpose_category'].count()*100))+'%')
print ('Процент невозвратов в категории "Образование" :', 
       str(int(education_debt.count()/education['purpose_category'].count()*100))+'%')
print ('Процент невозвратов в категории "Свадьба"     :', 
       str(int(wedding_debt.count()/wedding['purpose_category'].count()*100))+'%')

Процент невозвратов в категории "Недвижимость": 7%
Процент невозвратов в категории "Автомобили"  : 9%
Процент невозвратов в категории "Образование" : 9%
Процент невозвратов в категории "Свадьба"     : 7%


Для общего представления с помощью функции value_counts() смотрим сколько кредитов берется на куждую из категорий целей.
Затем в новые переменные записываем строки с одной из 4 категорий цели взятия кредита.
И наконец находим количество имевших задолжности и их отношение ко всему количеству в каждой из категорий.

##### Вывод: 
После проделанных расчетов для каждой категории целей взятия кредита можно с уверенностью сказать, что цель взятия кредита не влияет на возврат кредита в срок, так как проценты невозвратов для каждой категории очень схожи и максимальная разница между ними не превышает 2%.

## Общий вывод:

Были проверены 4 гипотезы установлено:

1. Зависимости между количеством детей и возвратом кредита в срок нет
2. Зависимости между семейным положением и возвратом кредита в срок нет
3. Зависимости между уровнем дохода и возвратом кредита в срок нет
4. Разные цели кредита не влияют на его возврат в срок

Ни одна из 4 представленных гипотез не подтвердилась. 

Были выявлены некие мизерные зависимости, но сложно говорить о каких-то закономерностях при столь малом отличии одних показателей от других. Каждый раз анализ данных показывал приблизительное сходство между всеми категориями сравнения, что не давало возможности назвать гипотезу правдивой или выявить какую-никакую значимую закономерность. 